In [371]:
import pandas as pd
import sklearn
!pip install sklearn_crfsuite
import sklearn_crfsuite
from nltk.tokenize import word_tokenize
import re
from sklearn.model_selection import KFold 
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers

In [372]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [373]:
#read datasets
words = open("sample_data/NE.ma.txt", "r")
text = open("sample_data/NE.txt", "r")

words = words.read()
#text = text.read()

In [374]:
#Extract information
index = 1
labels = list()
for i in text.readlines():
  tuple_ = re.findall("<b_enamex TYPE=\"(\w*)\">([\w ]+)<e_enamex>", i)  #([\w ]+)
  if tuple_:
    for j in tuple_: 
      labels.append((index, j[0], j[1]))
  
  index = index + 1
labels

[(1, 'ORGANIZATION', 'POZİTİF ve Açık Radyo'),
 (1, 'LOCATION', 'İstanbul'),
 (4, 'LOCATION', 'Askeri Müze ve Kültür Sitesi'),
 (5, 'LOCATION', 'Anadolu'),
 (7, 'LOCATION', 'VENEDİK'),
 (8, 'PERSON', 'Akira Kurosawa'),
 (10, 'PERSON', 'Steven Spielberg'),
 (10, 'PERSON', 'Ryan'),
 (10, 'PERSON', 'Peter Weir'),
 (10, 'PERSON', 'Truman'),
 (10, 'PERSON', 'Stanley Kubrick'),
 (10, 'PERSON', 'Taviani'),
 (10, 'PERSON', 'Woody Allen'),
 (11, 'PERSON', 'Valeria Golino'),
 (11, 'PERSON', 'Maria Grazia Cucinotta'),
 (11, 'PERSON', 'Asia Argento'),
 (11, 'LOCATION', 'Lido'),
 (14, 'PERSON', 'Fernando Solanas'),
 (14, 'PERSON', 'Eric Rohmer'),
 (14, 'LOCATION', 'İran'),
 (14, 'PERSON', 'Mohsen Makhmalbef'),
 (14, 'LOCATION', 'Avrupa'),
 (14, 'PERSON', 'Yves Angelo'),
 (14, 'PERSON', 'Emir Kusturica'),
 (14, 'PERSON', 'Gianni Amelio'),
 (14, 'PERSON', 'Francesca Archibugi'),
 (14, 'PERSON', 'Daniele Luchetti'),
 (14, 'PERSON', 'Lucien Pintilie'),
 (14, 'LOCATION', 'İspanya'),
 (14, 'PERSON', 'Jul

In [375]:
##IO Tagging
#Seperate entities ==> (row number, label, word)
"""
IOLabels = list()
for i in labels:
  for j in i[2].split(" "):
    if i[1] == "ORGANIZATION":
        IOLabels.append((i[0], "I-ORG", j))

    elif i[1] == "PERSON":
        IOLabels.append((i[0],"I-PER", j))

    elif i[1] == "LOCATION":
        IOLabels.append((i[0], "I-LOC", j))
        """

'\nIOLabels = list()\nfor i in labels:\n  for j in i[2].split(" "):\n    if i[1] == "ORGANIZATION":\n        IOLabels.append((i[0], "I-ORG", j))\n\n    elif i[1] == "PERSON":\n        IOLabels.append((i[0],"I-PER", j))\n\n    elif i[1] == "LOCATION":\n        IOLabels.append((i[0], "I-LOC", j))\n        '

In [376]:
##IOB Tagging
#Seperate entities ==> (row number, label, word)

IOLabels = list()
for i in labels:
  check = True
  for j in i[2].split(" "):
    if i[1] == "ORGANIZATION":
      if check is True:
        IOLabels.append((i[0], "B-ORG", j))
        check = False
      else:
        IOLabels.append((i[0], "I-ORG", j))

    elif i[1] == "PERSON":
      if check is True:
        IOLabels.append((i[0],"B-PER", j))
        check = False
      else:
        IOLabels.append((i[0],"I-PER", j))

    elif i[1] == "LOCATION":
      if check is True:
        IOLabels.append((i[0], "B-LOC", j))
        check = False
      else:
        IOLabels.append((i[0], "I-LOC", j))
        


In [377]:
IOLabels

[(1, 'B-ORG', 'POZİTİF'),
 (1, 'I-ORG', 've'),
 (1, 'I-ORG', 'Açık'),
 (1, 'I-ORG', 'Radyo'),
 (1, 'B-LOC', 'İstanbul'),
 (4, 'B-LOC', 'Askeri'),
 (4, 'I-LOC', 'Müze'),
 (4, 'I-LOC', 've'),
 (4, 'I-LOC', 'Kültür'),
 (4, 'I-LOC', 'Sitesi'),
 (5, 'B-LOC', 'Anadolu'),
 (7, 'B-LOC', 'VENEDİK'),
 (8, 'B-PER', 'Akira'),
 (8, 'I-PER', 'Kurosawa'),
 (10, 'B-PER', 'Steven'),
 (10, 'I-PER', 'Spielberg'),
 (10, 'B-PER', 'Ryan'),
 (10, 'B-PER', 'Peter'),
 (10, 'I-PER', 'Weir'),
 (10, 'B-PER', 'Truman'),
 (10, 'B-PER', 'Stanley'),
 (10, 'I-PER', 'Kubrick'),
 (10, 'B-PER', 'Taviani'),
 (10, 'B-PER', 'Woody'),
 (10, 'I-PER', 'Allen'),
 (11, 'B-PER', 'Valeria'),
 (11, 'I-PER', 'Golino'),
 (11, 'B-PER', 'Maria'),
 (11, 'I-PER', 'Grazia'),
 (11, 'I-PER', 'Cucinotta'),
 (11, 'B-PER', 'Asia'),
 (11, 'I-PER', 'Argento'),
 (11, 'B-LOC', 'Lido'),
 (14, 'B-PER', 'Fernando'),
 (14, 'I-PER', 'Solanas'),
 (14, 'B-PER', 'Eric'),
 (14, 'I-PER', 'Rohmer'),
 (14, 'B-LOC', 'İran'),
 (14, 'B-PER', 'Mohsen'),
 (14, 'I-

In [378]:
# Extract information with regex ==> (row number, word, features)
features = re.findall("(\d+) ([\w+\']+) ([\w+\+]+)", words)  #(\d+) (\w+) 
features

[('1', 'Müzik', 'müzik+Noun+A3sg+Pnon+Nom'),
 ('1', "Şenliği'ne", 'Şenlik+Noun+Prop+A3sg+P3sg+Dat'),
 ('1', 'hazırlanın', 'hazırla+Verb'),
 ('1', 'POZİTİF', 'pozitif+Adj'),
 ('1', 've', 've+Conj'),
 ('1', 'Açık', 'açık+Adj'),
 ('1', 'Radyo', 'radyo+Noun+A3sg+Pnon+Nom'),
 ('1', 'işbirliğiyle', 'işbirliği+Noun+A3sg+Pnon+Ins'),
 ('1', 'düzenlenecek', 'düzenle+Verb'),
 ('1', 'olan', 'ol+Verb+Pos'),
 ('1', 'İstanbul', 'İstanbul+Noun+Prop+A3sg+Pnon+Nom'),
 ('1', 'Müzik', 'müzik+Noun+A3sg+Pnon+Nom'),
 ('1', 'Şenliği', 'şenlik+Noun+A3sg+P3sg+Nom'),
 ('1', '2', '2+Num+Card'),
 ('1', 'müzikseverlere', 'müziksever+Noun+A3pl+Pnon+Dat'),
 ('1', 'Aralık', 'aralık+Noun+A3sg+Pnon+Nom'),
 ('1', 'ayında', 'ay+Noun+A3sg+P3sg+Loc'),
 ('1', 'merhaba', 'merhaba+Noun+A3sg+Pnon+Nom'),
 ('1', 'demeye', 'de+Verb+Pos'),
 ('1', 'hazırlanıyor', 'hazırla+Verb'),
 ('2', 'Tek', 'tek+Adj'),
 ('2', 'çatı', 'çatı+Noun+A3sg+Pnon+Nom'),
 ('2', 'altında', 'alt+Noun+A3sg+P3sg+Loc'),
 ('2', 'dokuz', 'dokuz+Num+Card'),
 ('2',

#Preprocessing

In [379]:
#Create Dataframe
df_features = pd.DataFrame(features, columns=["Row", "Words", "Feature"])
df_features["Row"] = pd.to_numeric(df_features["Row"])

In [380]:
#Seperate features
df_features["Features2"]= df_features["Feature"].str.split("+")

In [381]:
df_features

,Row,Words,Feature,Features2
0,1,Müzik,müzik+Noun+A3sg+Pnon+Nom,"[müzik, Noun, A3sg, Pnon, Nom]"
1,1,Şenliği'ne,Şenlik+Noun+Prop+A3sg+P3sg+Dat,"[Şenlik, Noun, Prop, A3sg, P3sg, Dat]"
2,1,hazırlanın,hazırla+Verb,"[hazırla, Verb]"
3,1,POZİTİF,pozitif+Adj,"[pozitif, Adj]"
4,1,ve,ve+Conj,"[ve, Conj]"
...,...,...,...,...
143702,9999,bir,bir+Det,"[bir, Det]"
143703,9999,ölçüde,ölçü+Noun+A3sg+Pnon+Loc,"[ölçü, Noun, A3sg, Pnon, Loc]"
143704,9999,prim,prim+Noun+A3sg+Pnon+Nom,"[prim, Noun, A3sg, Pnon, Nom]"
143705,9999,kazandırdı,kazan+Verb,"[kazan, Verb]"


In [382]:
#Create dataframe
df_labels = pd.DataFrame(IOLabels, columns=["Row", "Label", "Words"])
df_labels

,Row,Label,Words
0,1,B-ORG,POZİTİF
1,1,I-ORG,ve
2,1,I-ORG,Açık
3,1,I-ORG,Radyo
4,1,B-LOC,İstanbul
...,...,...,...
16868,9995,B-ORG,DSP
16869,9996,B-ORG,DSP
16870,9997,B-ORG,DSP
16871,9998,B-ORG,DSP


In [383]:
#Merge dataframes witf based on df_features dataset
merged_df = df_features.merge(df_labels, on = ["Row", "Words"], how="left")

In [384]:
merged_df["Label"].fillna("O", inplace=True)

In [385]:
merged_df

,Row,Words,Feature,Features2,Label
0,1,Müzik,müzik+Noun+A3sg+Pnon+Nom,"[müzik, Noun, A3sg, Pnon, Nom]",O
1,1,Şenliği'ne,Şenlik+Noun+Prop+A3sg+P3sg+Dat,"[Şenlik, Noun, Prop, A3sg, P3sg, Dat]",O
2,1,hazırlanın,hazırla+Verb,"[hazırla, Verb]",O
3,1,POZİTİF,pozitif+Adj,"[pozitif, Adj]",B-ORG
4,1,ve,ve+Conj,"[ve, Conj]",I-ORG
...,...,...,...,...,...
145159,9999,bir,bir+Det,"[bir, Det]",O
145160,9999,ölçüde,ölçü+Noun+A3sg+Pnon+Loc,"[ölçü, Noun, A3sg, Pnon, Loc]",O
145161,9999,prim,prim+Noun+A3sg+Pnon+Nom,"[prim, Noun, A3sg, Pnon, Nom]",O
145162,9999,kazandırdı,kazan+Verb,"[kazan, Verb]",O


In [386]:
#Add Features
merged_df["word.lower"] = merged_df["Words"].str.lower()
merged_df["postag"] = merged_df['Features2'].str[1]
merged_df["Root"] = merged_df['Features2'].str[0]
merged_df["word.istitle"] = merged_df["Words"].str.istitle()
merged_df["word.isupper"] = merged_df["Words"].str.isupper()
merged_df["word.isdigit"] = merged_df["Words"].str.isdigit()
merged_df["pnon"] = merged_df["Feature"].str.contains("Pnon")
merged_df["nom"] = merged_df["Feature"].str.contains("Nom")
merged_df["inf"] = merged_df["Features2"].str[2:]  

In [387]:
#Gazzetteer from 1st project
organizations = ["Türkiye Cumhuriyeti", "TBMM", "Türkiye Büyük Millet Meclisi", "Cumhurbaşkanlığı",
                "Bakanlık", "ABD", "Amerika Birleşik Devletleri", "NATO", "Çin Halk Cumhuriyeti" ,
                "Merkez Bankası", "CHP", "MHP", "AKP", "AK Parti", "HDP", "İYİ Parti", "DEVA Partisi",
                "Gelecek Partisi", "HADEP", "ANAP", "DSP", "Yargıtay", "Sayıştay", "Danıştay", "YÖK", "Yüksek Öğretim Kurumu",
                "YSK", "Yüksek Seçim Kurumu", "Hakimler ve Savcılar Yüksek Kurulu", "Hakimler ve Savcılar Kurulu", "HSK",
                 "HSYK", "Milli Güvenlik Kurulu", "Avrupa Birliği", "AB", "Mehmetçik Vakfı",
                "Pegasus Havayolları", "Türk Hava Yolları", "THY", "Ülker", "Akbank", "İş Bankası", "Yapı Kredi Bankası",
                "Ziraat Bankası", "Dünya Bankası", "Halk Bankası", "Tüpraş", "TPAO", "Türkiye Petrolleri Anonim Ortaklığı",
                "Vodafone", "Migros", "Tofaş", "Mercedes" ,"Doğuş Otomotiv", "Enerjisa", "Ereğli Demir Çelik", "Turkcell",
                "Arçelik", "Türk Telekom", "Shell", "Ford Otosan", "BİM", "OPET",
                "Apple", "Samsung", "Tesla", "IBM", "Google", "Facebook" , "Intel", "Microsoft", "Sony", "TRT",
                "Üniversitesi", "Holding", "Vakfı", "Federasyonu", "Şirket", "Enstitüsü", "Kurumu", "Bankası", "Kurumu",
                "Bakanlığı", "Köyü", "Dağı", "Mahallesi", "Sokağı", "Köprüsü", "Sarayı", "Mezarlığı", "Futbol Takımı",
                "Müzesi", "Partisi",  "Belediyesi", "Büyükşehir Belediyesi", "Marinası", "Kulübü", "Gazetesi", "Festivali",
                "Sitesi", "Apartmanı", "Konağı", "Köşkü", "Külliyesi", "Radyo", "Barosu", "Karayolu"]

In [388]:
#Removed
#merged_df["is_organization"] =  merged_df["Root"].apply(lambda x: True if x in organizations else False) 

In [389]:
def wordshape(text):
    import re
    t1 = re.sub('[A-ZĞÜŞİÖÇ]', 'X',text)
    t2 = re.sub('[a-zığüşiöç]', 'x', t1)
    return re.sub('[0-9]', 'd', t2)

In [390]:
merged_df["word.shape"] = merged_df["Words"].apply(lambda x: wordshape(x)) 

In [391]:
merged_df["Prev_Label"] = merged_df['Label'].shift(1)
merged_df["Prev_Label"].fillna("O", inplace=True)

merged_df["Prev_istitle"] = merged_df['word.istitle'].shift(1)
merged_df["Prev_istitle"].fillna(False, inplace=True)

merged_df["Prev_isupper"] = merged_df['word.isupper'].shift(1)
merged_df["Prev_isupper"].fillna(False, inplace=True)

merged_df["Prev_lower"] = merged_df['word.lower'].shift(1)
merged_df["Prev_lower"].fillna("", inplace=True)

merged_df["Prev_postag"] = merged_df["postag"].shift(1)
merged_df["Prev_postag"].fillna("Noun", inplace=True)

In [392]:
merged_df

,Row,Words,Feature,Features2,Label,word.lower,postag,Root,word.istitle,word.isupper,word.isdigit,pnon,nom,inf,word.shape,Prev_Label,Prev_istitle,Prev_isupper,Prev_lower,Prev_postag
0,1,Müzik,müzik+Noun+A3sg+Pnon+Nom,"[müzik, Noun, A3sg, Pnon, Nom]",O,müzik,Noun,müzik,True,False,False,True,True,"[A3sg, Pnon, Nom]",Xxxxx,O,False,False,,Noun
1,1,Şenliği'ne,Şenlik+Noun+Prop+A3sg+P3sg+Dat,"[Şenlik, Noun, Prop, A3sg, P3sg, Dat]",O,şenliği'ne,Noun,Şenlik,False,False,False,False,False,"[Prop, A3sg, P3sg, Dat]",Xxxxxxx'xx,O,True,False,müzik,Noun
2,1,hazırlanın,hazırla+Verb,"[hazırla, Verb]",O,hazırlanın,Verb,hazırla,False,False,False,False,False,[],xxxxxxxxxx,O,False,False,şenliği'ne,Noun
3,1,POZİTİF,pozitif+Adj,"[pozitif, Adj]",B-ORG,pozi̇ti̇f,Adj,pozitif,False,True,False,False,False,[],XXXXXXX,O,False,False,hazırlanın,Verb
4,1,ve,ve+Conj,"[ve, Conj]",I-ORG,ve,Conj,ve,False,False,False,False,False,[],xx,B-ORG,False,True,pozi̇ti̇f,Adj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145159,9999,bir,bir+Det,"[bir, Det]",O,bir,Det,bir,False,False,False,False,False,[],xxx,O,False,False,dsp'ye,Noun
145160,9999,ölçüde,ölçü+Noun+A3sg+Pnon+Loc,"[ölçü, Noun, A3sg, Pnon, Loc]",O,ölçüde,Noun,ölçü,False,False,False,True,False,"[A3sg, Pnon, Loc]",xxxxxx,O,False,False,bir,Det
145161,9999,prim,prim+Noun+A3sg+Pnon+Nom,"[prim, Noun, A3sg, Pnon, Nom]",O,prim,Noun,prim,False,False,False,True,True,"[A3sg, Pnon, Nom]",xxxx,O,False,False,ölçüde,Noun
145162,9999,kazandırdı,kazan+Verb,"[kazan, Verb]",O,kazandırdı,Verb,kazan,False,False,False,False,False,[],xxxxxxxxxx,O,False,False,prim,Noun


In [393]:
#Prepare train set ==> list of list of dictionaries 
df_features = merged_df[["word.lower", "postag", "word.istitle", "word.isupper", "word.isdigit", "pnon", "nom", "inf", "word.shape", "Prev_Label", "Prev_istitle", "Prev_isupper", "Prev_lower", "Prev_postag", "Row"]].groupby(by=["Row"])

x_train = list()

for i in range(len(df_features.groups)):

  x_train.append(df_features.get_group(i+1)[["word.lower", "postag", "word.istitle", "word.isupper", "word.isdigit", "pnon", "nom", "inf", "word.shape", "Prev_Label", "Prev_istitle", "Prev_isupper", "Prev_lower", "Prev_postag"]].to_dict("records"))

In [394]:
#Prepare label set ==> list of list of labels
df_labels = merged_df[["Label", "Row"]].groupby(by=["Row"])

y_train = list()

for i in range(len(df_labels.groups)):

  y_train.append(df_labels.get_group(i+1)["Label"].to_list())

#5 Fold Split

In [395]:
# 5 folds split
x_train_1 = list()
x_train_2 = list()
x_train_3 = list()
x_train_4 = list()
x_train_5 = list()

y_train_1 = list()
y_train_2 = list()
y_train_3 = list()
y_train_4 = list()
y_train_5 = list()

for i in range(len(x_train)):
  if i % 5 == 0:
    x_train_1.append(x_train[i])
    y_train_1.append(y_train[i])
  elif i % 5 == 1:
    x_train_2.append(x_train[i])
    y_train_2.append(y_train[i])
  elif i % 5 == 2:
    x_train_3.append(x_train[i])
    y_train_3.append(y_train[i])
  elif i % 5 == 3:
    x_train_4.append(x_train[i])
    y_train_4.append(y_train[i])
  elif i % 5 == 4:
    x_train_5.append(x_train[i])
    y_train_5.append(y_train[i])

#Training

In [396]:
#hyper parameters
c1 = 0.1
c2 = 0.1
max_iter = 150
all_poss_trans = True

In [397]:
#First fold is test
crf1_x = list()
crf1_y = list()

crf1_x.extend(x_train_2)
crf1_x.extend(x_train_3)
crf1_x.extend(x_train_4)
crf1_x.extend(x_train_5)

crf1_y.extend(y_train_2)
crf1_y.extend(y_train_3)
crf1_y.extend(y_train_4)
crf1_y.extend(y_train_5)


crf1 = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1,
    c2=c2,
    max_iterations=max_iter,
    all_possible_transitions=all_poss_trans
)

crf1.fit(crf1_x, crf1_y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=150,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [398]:
crf2_x = list()
crf2_y = list()

crf2_x.extend(x_train_1)
crf2_x.extend(x_train_3)
crf2_x.extend(x_train_4)
crf2_x.extend(x_train_5)

crf2_y.extend(y_train_1)
crf2_y.extend(y_train_3)
crf2_y.extend(y_train_4)
crf2_y.extend(y_train_5)

#Second fold is test
crf2 = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1,
    c2=c2,
    max_iterations=max_iter,
    all_possible_transitions=all_poss_trans
)

crf2.fit(crf2_x, crf2_y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=150,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [399]:
crf3_x = list()
crf3_y = list()

crf3_x.extend(x_train_1)
crf3_x.extend(x_train_2)
crf3_x.extend(x_train_4)
crf3_x.extend(x_train_5)

crf3_y.extend(y_train_1)
crf3_y.extend(y_train_2)
crf3_y.extend(y_train_4)
crf3_y.extend(y_train_5)

#Third fold is test
crf3 = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1,
    c2=c2,
    max_iterations=max_iter,
    all_possible_transitions=all_poss_trans
)

crf3.fit(crf3_x, crf3_y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=150,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [400]:
crf4_x = list()
crf4_y = list()

crf4_x.extend(x_train_1)
crf4_x.extend(x_train_2)
crf4_x.extend(x_train_3)
crf4_x.extend(x_train_5)

crf4_y.extend(y_train_1)
crf4_y.extend(y_train_2)
crf4_y.extend(y_train_3)
crf4_y.extend(y_train_5)

#Fourth fold is test
crf4 = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1,
    c2=c2,
    max_iterations=max_iter,
    all_possible_transitions=all_poss_trans
)

crf4.fit(crf4_x, crf4_y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=150,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [401]:
crf5_x = list()
crf5_y = list()

crf5_x.extend(x_train_1)
crf5_x.extend(x_train_2)
crf5_x.extend(x_train_3)
crf5_x.extend(x_train_4)

crf5_y.extend(y_train_1)
crf5_y.extend(y_train_2)
crf5_y.extend(y_train_3)
crf5_y.extend(y_train_4)

#Fifth fold is test
crf5 = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1,
    c2=c2,
    max_iterations=max_iter,
    all_possible_transitions=all_poss_trans
)

crf5.fit(crf5_x, crf5_y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=150,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

#Evaluation

In [402]:
labels = list(crf1.classes_)
labels.remove('O')
labels

['B-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'I-LOC']

In [403]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
sorted_labels

['B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER']

In [404]:
pred1 = crf1.predict(x_train_1)
crf1_f1_score = metrics.flat_f1_score(y_train_1, pred1, average='weighted', labels=labels)
crf1_f1_score

0.8588449094256038

In [405]:
crf1_acc = metrics.flat_accuracy_score(y_train_1, pred1)
crf1_acc

0.9810167862414609

In [406]:
crf1_matrix = metrics.flat_classification_report(
    y_train_1, pred1, digits=3, labels=sorted_labels)
print(crf1_matrix)

              precision    recall  f1-score   support

       B-LOC      0.883     0.855     0.869       550
       I-LOC      0.702     0.688     0.695        48
       B-ORG      0.921     0.789     0.850       535
       I-ORG      0.858     0.852     0.855       283
       B-PER      0.907     0.842     0.874       920
       I-PER      0.823     0.864     0.843       339

   micro avg      0.884     0.835     0.859      2675
   macro avg      0.849     0.815     0.831      2675
weighted avg      0.886     0.835     0.859      2675



In [407]:
pred2 = crf2.predict(x_train_2)
crf2_f1_score = metrics.flat_f1_score(y_train_2, pred2, average='weighted', labels=labels)
crf2_f1_score

0.8588189304668317

In [408]:
crf2_acc = metrics.flat_accuracy_score(y_train_2, pred2)
crf2_acc

0.9803311112738994

In [409]:
crf2_matrix = metrics.flat_classification_report(
    y_train_2, pred2, digits=3, labels=sorted_labels)
print(crf2_matrix)

              precision    recall  f1-score   support

       B-LOC      0.880     0.872     0.876       538
       I-LOC      0.720     0.581     0.643        31
       B-ORG      0.926     0.805     0.861       467
       I-ORG      0.878     0.902     0.890       255
       B-PER      0.904     0.816     0.858       934
       I-PER      0.804     0.859     0.830       368

   micro avg      0.882     0.837     0.859      2593
   macro avg      0.852     0.806     0.826      2593
weighted avg      0.884     0.837     0.859      2593



In [410]:
pred3 = crf3.predict(x_train_3)
crf3_f1_score = metrics.flat_f1_score(y_train_3, pred3, average='weighted', labels=labels)
crf3_f1_score

0.8692276128821377

In [411]:
crf3_acc = metrics.flat_accuracy_score(y_train_3, pred3)
crf3_acc

0.9819708654670094

In [412]:
crf3_matrix = metrics.flat_classification_report(
    y_train_3, pred3, digits=3, labels=sorted_labels)
print(crf3_matrix)

              precision    recall  f1-score   support

       B-LOC      0.913     0.886     0.899       606
       I-LOC      0.893     0.543     0.676        46
       B-ORG      0.931     0.837     0.881       497
       I-ORG      0.946     0.868     0.905       281
       B-PER      0.898     0.841     0.869       982
       I-PER      0.769     0.837     0.802       369

   micro avg      0.892     0.848     0.869      2781
   macro avg      0.891     0.802     0.839      2781
weighted avg      0.895     0.848     0.869      2781



In [413]:
pred4 = crf4.predict(x_train_4)
crf4_f1_score = metrics.flat_f1_score(y_train_4, pred4, average='weighted', labels=labels)
crf4_f1_score

0.8452159264628389

In [414]:
crf4_acc = metrics.flat_accuracy_score(y_train_4, pred4)
crf4_acc

0.979193714837378

In [415]:
crf4_matrix = metrics.flat_classification_report(
    y_train_4, pred4, digits=3, labels=sorted_labels)
print(crf4_matrix)

              precision    recall  f1-score   support

       B-LOC      0.895     0.802     0.846       646
       I-LOC      0.759     0.759     0.759        83
       B-ORG      0.839     0.761     0.798       548
       I-ORG      0.883     0.866     0.875       262
       B-PER      0.920     0.847     0.882      1011
       I-PER      0.790     0.832     0.811       358

   micro avg      0.874     0.818     0.845      2908
   macro avg      0.848     0.811     0.828      2908
weighted avg      0.875     0.818     0.845      2908



In [416]:
pred5 = crf5.predict(x_train_5)
crf5_f1_score = metrics.flat_f1_score(y_train_5, pred5, average='weighted', labels=labels)
crf5_f1_score

0.841457311595839

In [417]:
crf5_acc = metrics.flat_accuracy_score(y_train_5, pred5)
crf5_acc

0.9774706101568588

In [418]:
crf5_matrix = metrics.flat_classification_report(
    y_train_5, pred5, digits=3, labels=sorted_labels)
print(crf5_matrix)

              precision    recall  f1-score   support

       B-LOC      0.887     0.856     0.872       606
       I-LOC      0.841     0.757     0.797        70
       B-ORG      0.907     0.787     0.843       483
       I-ORG      0.824     0.838     0.831       241
       B-PER      0.898     0.801     0.847      1088
       I-PER      0.778     0.809     0.793       398

   micro avg      0.871     0.814     0.841      2886
   macro avg      0.856     0.808     0.830      2886
weighted avg      0.873     0.814     0.841      2886



In [419]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf1.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf1.transition_features_).most_common()[-20:])

Top likely transitions:
B-ORG  -> I-ORG   6.973468
B-PER  -> I-PER   6.833334
I-ORG  -> I-ORG   5.939585
I-PER  -> I-PER   4.392565
B-ORG  -> I-PER   4.333053
I-LOC  -> I-LOC   3.669786
I-PER  -> I-ORG   3.182178
B-LOC  -> I-LOC   2.999423
O      -> I-PER   2.256619
B-ORG  -> B-ORG   2.105541
B-LOC  -> B-LOC   1.436637
B-ORG  -> B-LOC   1.418811
O      -> I-ORG   1.393822
O      -> B-LOC   1.389210
B-PER  -> B-PER   1.373591
O      -> B-ORG   1.278477
O      -> B-PER   1.085725
B-PER  -> B-ORG   1.052337
I-LOC  -> B-LOC   0.922699
I-ORG  -> B-ORG   0.650670

Top unlikely transitions:
I-LOC  -> B-ORG   -0.323617
I-PER  -> B-PER   -0.389807
I-ORG  -> I-PER   -0.407272
I-LOC  -> O       -0.565836
I-ORG  -> I-LOC   -0.578184
B-PER  -> O       -0.641299
B-LOC  -> I-PER   -0.666581
B-ORG  -> O       -0.698365
I-ORG  -> B-LOC   -0.699812
I-ORG  -> O       -0.829237
B-ORG  -> I-LOC   -0.916291
B-PER  -> I-ORG   -1.123873
B-ORG  -> B-PER   -1.146085
I-PER  -> I-LOC   -1.332798
B-LOC  -> O      

#Average Scores

In [420]:
avg_f1_score = (crf1_f1_score + crf2_f1_score + crf3_f1_score + crf4_f1_score + crf5_f1_score)/5
avg_f1_score

0.8547129381666501

In [421]:
avg_acc = (crf1_acc + crf2_acc + crf3_acc + crf4_acc + crf5_acc)/5
avg_acc

0.9799966175953212